In [1]:
#importing libraries
import requests
import os
import io
import random
import json
import re
import time
import logging
import asyncio
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langgraph.graph import  StateGraph, END
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from IPython.display import display, Image
from langchain_core.runnables import RunnableLambda
from typing import TypedDict, List
#from source.scripts import extract_search_queries, search_duckduckgo, fetch_full_text, create_sub_agents
from duckduckgo_search import DDGS
from typing import Dict, List, Optional
from db import ResearchDatabase
from source.utils import ResearchHelpFunctions

/workspace/speech-to-text-transcription/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#initializing the db
db = ResearchDatabase()
user_id = "user_725"
session_id = db.create_session(user_id= user_id, metadata = {"purpose": "football research"})

In [3]:
#initializing the help functions
assistant = ResearchHelpFunctions()

In [4]:
#importing the dotenv files
load_dotenv()

groqquiz_api_key = os.getenv("RESEARCH_QUESTIONS_GENERATOR")
groqstructure_api_key = os.getenv("RESEARCH_STRUCTURE_GENERATOR")
groqmanager_api_key = os.getenv("MANAGER_AGENT_RESEARCH")
groqworker1_api_key = os.getenv("WORKER_AGENT_1")
groqworker2_api_key = os.getenv("WORKER_AGENT_2")
groqworker3_api_key = os.getenv("WORKER_AGENT_3")
groqworker4_api_key = os.getenv("WORKER_AGENT_4")
groqworker5_api_key = os.getenv("WORKER_AGENT_5")
groqliteratureworker_api_key = os.getenv("LITERATURE_REVIEW_AGENT")
groqsub_agents_api_key = os.getenv("SUB_AGENT")
summary_model = os.getenv("SUMMARY_AGENT")

if not groqquiz_api_key:
    raise ValueError("Error loading API Key!")
if not groqstructure_api_key:
    raise ValueError("Error loading API Key!")
if not groqmanager_api_key:
    raise ValueError("Error loading API Key!")
if not groqworker1_api_key:
    raise ValueError("Error loading API Key!")
if not groqworker2_api_key:
    raise ValueError("Error loading API Key!")
if not groqworker3_api_key:
    raise ValueError("Error loading API Key!")
if not groqworker4_api_key:
    raise ValueError("Error loading API Key!")
if not groqworker5_api_key:
    raise ValueError("Error loading API Key!")
if not groqliteratureworker_api_key:
    raise ValueError("Error loading API Key!")
if not groqsub_agents_api_key:
    raise ValueError("Error loading API Key!")


In [5]:
#initializing the llms used

llm_query = ChatGroq(
    temperature = 0.7,
    api_key = groqquiz_api_key,
    model = "qwen-qwq-32b"
)
llm_structure = ChatGroq(
    temperature = 0.7,
    api_key = groqstructure_api_key,
    model = "qwen-qwq-32b"
)
llm_manager = ChatGroq(
    temperature = 0.7,
    api_key = groqmanager_api_key,
    model = "qwen-qwq-32b"
)
llm_worker1 = ChatGroq(
    temperature = 0.7,
    api_key = groqworker1_api_key,
    model = "qwen-qwq-32b"
)
llm_worker2 = ChatGroq(
    temperature = 0.7,
    api_key = groqworker2_api_key,
    model = "qwen-qwq-32b"
)
llm_worker3 = ChatGroq(
    temperature = 0.7,
    api_key = groqworker3_api_key,
    model = "qwen-qwq-32b"
)
llm_worker4 = ChatGroq(
    temperature = 0.7,
    api_key = groqworker4_api_key,
    model = "qwen-qwq-32b"
)
llm_worker5 = ChatGroq(
    temperature = 0.7,
    api_key = groqworker5_api_key,
    model = "qwen-qwq-32b"
)
llm_literature = ChatGroq(
    temperature = 0.7,
    api_key = groqliteratureworker_api_key,
    model = "qwen-qwq-32b"
)
summary_model = ChatGroq(
    temperature = 0.7,
    api_key = summary_model,
    model = "qwen-qwq-32b"
)

In [6]:
#An agent that creates multiple queries
def create_multiple_queries(query):
    prompt = ChatPromptTemplate.from_template("""
        <Goal>
    <Primary>
        To generate five distinct, research-quality search queries that directly address the user's topic. These queries should be formulated as someone would type them into a search engine when conducting thorough academic or professional research.
    </Primary>
</Goal>

<Instructions>
    <Instruction>
        1. Extract the exact concepts and keywords from the user's topic without introducing unrelated themes or expanding the scope unless explicitly requested.
    </Instruction>
    <Instruction>
        2. Preserve the user's specific terminology and focus. Do not substitute or generalize terms (e.g., if user specifies "happiness," do not change to "well-being" or "mental health").
    </Instruction>
    <Instruction>
        3. Create exactly five unique search queries that:
            - Center on the main keywords from the user's input
            - Include thoughtful variations that maintain the original focus
            - Represent different research angles within the stated scope
            - Are formatted as complete search terms someone would actually type
    </Instruction>
    <Instruction>
        4. Structure the output in this JSON format:
        ```json
        {{
            "topic": "USER_INPUT_TOPIC",
            "searchQueries": [
                "QUERY_1",
                "QUERY_2",
                "QUERY_3",
                "QUERY_4",
                "QUERY_5"
            ]
        }}
        ```
    </Instruction>
    <Instruction>
        5. Prioritize depth over breadth in query formulation, focusing on specific aspects that would yield substantive research material.
    </Instruction>
</Instructions>

<Examples>
    <Example>
        <UserInput>gut microbiome link to happiness</UserInput>
        <AgentOutput>
            {{
                "topic": "gut microbiome link to happiness",
                "searchQueries": [
                    "gut microbiome link to happiness research studies",
                    "gut microbial composition and happiness correlation",
                    "gut bacteria influence on happiness levels evidence",
                    "bacterial diversity and reported happiness direct link",
                    "latest findings gut microbiome happiness association"
                ]
            }}
        </AgentOutput>
    </Example>

    <Example>
        <UserInput>Creative writing with AI</UserInput>
        <AgentOutput>
            {{
                "topic": "Creative writing with AI",
                "searchQueries": [
                    "AI creative writing tools",
                    "artificial intelligence creative writing innovations",
                    "AI-assisted fiction writing techniques",
                    "exploring generative AI for creative writing",
                    "case studies AI-driven creative storytelling"
                ]
            }}
        </AgentOutput>
    </Example>

    <Example>
        <UserInput>quantum computing hardware</UserInput>
        <AgentOutput>
            {{
                "topic": "quantum computing hardware",
                "searchQueries": [
                    "quantum computing hardware architecture",
                    "latest developments in quantum processors",
                    "superconducting qubits hardware advancements",
                    "ion trap quantum computing devices",
                    "scalability challenges quantum hardware research"
                ]
            }}
        </AgentOutput>
    </Example>

    <Example>
        <UserInput>big data in retail</UserInput>
        <AgentOutput>
            {{
                "topic": "big data in retail",
                "searchQueries": [
                    "big data retail analytics best practices",
                    "customer insights from big data in retail",
                    "data-driven inventory management retail",
                    "big data solutions for e-commerce personalization",
                    "trends in big data adoption retail sector"
                ]
            }}
        </AgentOutput>
    </Example>

    <Example>
        <UserInput>self-driving car sensors</UserInput>
        <AgentOutput>
            {{
                "topic": "self-driving car sensors",
                "searchQueries": [
                    "lidar vs radar technology in self-driving cars",
                    "self-driving car sensors real-time data processing",
                    "sensor fusion methods for autonomous vehicles",
                    "advancements in self-driving car camera systems",
                    "emerging self-driving sensor technologies market analysis"
                ]
            }}
        </AgentOutput>
    </Example>
</Examples>
The user's topic is: {query}
    """)

    chain = prompt | llm_query
    response= chain.invoke({"query": query})
    queries = assistant.extract_search_queries(response.content)
    return queries


In [7]:
# agent to create a research structure
def create_researh_structure(question: str, generated_queries: list):
    """
    Used to create a research structure after the questions have been generated by the agent

    ARGS:
        question: Topic of the research
        generated_queries: List of generated research questions

    Output:
        The research structure
    """
    prompt = ChatPromptTemplate.from_template(
"""
<Goal>
<Primary>
To generate research-quality search queries organized into a structured academic format: Introduction, Background, Research Findings, Application, and Conclusion — based strictly on the five provided research questions. Each section should use the corresponding question as-is and create detailed search engine queries for it.
</Primary>
</Goal>

<Instructions>
<Instruction>
1. Use the exact wording of the five research questions provided by the model. Do not modify, generalize, or reinterpret them.
</Instruction>
<Instruction>
2. For each research question:
    - Place it in the section that best matches its focus (e.g., Introductory framing goes under Introduction; observed results or data-driven insights go under Research Findings, etc.).
    - Generate 3–5 distinct, thoughtful search queries for that question.
    - Each query should be framed as someone would type into an academic or professional search engine.
</Instruction>
<Instruction>
3. Do not create your own research questions. You must only work with the 5 provided.
</Instruction>
<Instruction>
4. Format the final output in this JSON structure:
json
{{
  "topic": "{question}",
  "organizedSearchQueries": {{
    "Introduction": {{
      "question": "Q1",
      "queries": ["...", "..."]
    }},
    "Background": {{
      "question": "Q2",
      "queries": ["...", "..."]
    }},
    "Research Findings": {{
      "question": "Q3",
      "queries": ["...", "..."]
    }},
    "Application": {{
      "question": "Q4",
      "queries": ["...", "..."]
    }},
    "Conclusion": {{
      "question": "Q5",
      "queries": ["...", "..."]
    }}
  }}
}}
""")
    chain = prompt | llm_structure
    response_structure = chain.invoke({
        "question": question,
        "generated_queries": generated_queries
    })
    research_structure = assistant.extract_search_queries(response_structure.content)
    
    return research_structure


In [8]:
#worker that handles introduction
async def introduction_worker(queries: List[str]):
    """
    This receives a list of questions from the manager agent and does research on them.

    ARGS: 
        queries (List[str]): List of questions from the manager agent

    RETURNS: 
        str: The compiled introduction section
    """
    sub_agent = assistant.create_sub_agents(name = "Introduction_subworker")
    summaries = []
    introduction_links = [] 
    for i, query in enumerate(queries):
        print(f"Query {i+1}: {query}")
        try:
            summary = await sub_agent.ainvoke(query)
            print("Summary received:", summary)
            introduction_links.extend(summary.get('sources', []))
            summaries.append(f"Query {i+1} Summary:\n{summary['result']}\n")
        except Exception as e:
            summaries.append(f"Query {i+1} failed: {e}")

    prompt = ChatPromptTemplate.from_template(
        """
You are an academic writing assistant.

Your task is to synthesize the following research questions and their corresponding answers into a well-written **introduction section** for a research paper.

Research Questions:
{queries}

Answers:
{summaries}

Instructions:
- Combine and integrate the answers into a logically flowing introduction.
- Use the research questions and their answers to guide the structure and content.
- Add smooth transitions, context, and clarifying sentences to ensure the narrative flows well.
- Identify and fill any gaps in the content using general academic knowledge (without fabricating specific facts).
- Ensure the introduction is informative, engaging, and appropriate for an academic audience.
- Do not list the questions or answers separately—blend them into a unified and flowing narrative.
- Clearly introduce the topic, highlight its importance, and set up the background and rationale for the rest of the paper.
- The final output should be a polished **introduction section** that sets the stage for the entire research paper.

Now, write the introduction section:
"""
    )
    llm = llm_worker1
    chain = prompt | llm
    introduction = await chain.ainvoke({'queries':queries,
                               'summaries':summaries})
    introduction = re.sub(r'<think>.*?</think>', '', introduction.content, flags=re.DOTALL)
    return {'introduction_findings': introduction, 'introduction_sources': introduction_links}




In [9]:
#worker that handles introduction
async def Background_worker(queries: List[str]):
    """
    This receives a list of questions from the manager agent and does research on them.

    ARGS: 
        queries (List[str]): List of questions from the manager agent

    RETURNS: 
        str: The compiled Background section
    """
    sub_agent = assistant.create_sub_agents(name = "Background_subworker")
    summaries = []
    background_links = []
    for i, query in enumerate(queries):
        print(f"Query {i+1}: {query}")
        try:
            summary = await sub_agent.ainvoke(query)
            print("Summary received:", summary)
            background_links_links.extend(summary.get('sources', []))
            summaries.append(f"Query {i+1} Summary:\n{summary['result']}\n")
        except Exception as e:
            summaries.append(f"Query {i+1} failed: {e}")

    prompt = ChatPromptTemplate.from_template(
        """
You are an academic writing assistant.

Your task is to synthesize the following research questions and their corresponding answers into a well-written **background section** for a research paper.

Research Questions:
{queries}

Answers:
{summaries}

Instructions:
- Combine and integrate the answers into a logically flowing background section.
- Use the research questions and their answers to guide the structure and content.
- Add smooth transitions, context, and clarifying sentences to ensure the narrative flows well.
- Identify and fill any gaps in the content using general academic knowledge (without fabricating specific facts).
- Ensure the background is informative, engaging, and appropriate for an academic audience.
- Do not list the questions or answers separately—blend them into a unified and flowing narrative.
- Clearly provide the necessary context, foundational information, and the current state of knowledge that justifies and supports the research.
- The final output should be a polished **background section** that prepares readers for the rest of the paper.

Now, write the background section:
"""
    )
    llm = llm_worker2
    chain = prompt | llm
    background = await chain.ainvoke({'queries':queries,
                               'summaries':summaries})
    background = re.sub(r'<think>.*?</think>', '', background.content, flags=re.DOTALL)
    return {'background_findings': background, 'background_sources': background_links}



In [10]:
#worker that handles introduction
async def research_findings_worker(queries: List[str]):
    """
    This receives a list of questions from the manager agent and does research on them.

    ARGS: 
        queries (List[str]): List of questions from the manager agent

    RETURNS: 
        str: The compiled research findings section
    """
    sub_agent = assistant.create_sub_agents(name = "research_findings_subworker")
    summaries = []
    research_findings_links = []
    for i, query in enumerate(queries):
        print(f"Query {i+1}: {query}")
        try:
            summary = await sub_agent.ainvoke(query)
            print("Summary received:", summary)
            research_findings_links.extend(summary.get('sources', []))
            summaries.append(f"Query {i+1} Summary:\n{summary['result']}\n")
        except Exception as e:
            summaries.append(f"Query {i+1} failed: {e}")

    prompt = ChatPromptTemplate.from_template(
  """
You are an academic writing assistant.

Your task is to synthesize the following research questions and their corresponding answers into a well-written **Research Findings** section for a research paper.

Research Questions:
{queries}

Answers:
{summaries}

Instructions:
- Integrate the answers into a coherent and informative Research Findings section.
- Focus on discussing key studies, data, case examples, or findings related to the topic—whether scientific, academic, or practical.
- Use the research questions and answers as a foundation, but enrich the section with logical flow, context, and broader relevance.
- Include comparisons, insights, or patterns observed across the studies when appropriate.
- Fill any content gaps using general academic knowledge (without fabricating specific data).
- Maintain a professional, objective tone suitable for academic writing.
- Do not list the questions or answers separately—blend them into a unified, evidence-backed narrative.
- The final output should be a strong **Research Findings** section that demonstrates the current state of research on the topic.

Now, write the Research Findings section:
"""
    )
    llm = llm_worker3
    chain = prompt | llm
    research_findings = await chain.ainvoke({'queries':queries,
                               'summaries':summaries})
    research_findings= re.sub(r'<think>.*?</think>', '', research_findings.content, flags=re.DOTALL)
    return {'research_findings':research_findings, 'research_sources': research_findings_links}



In [11]:
#worker that handles introduction
async def application_worker(queries: List[str]):
    """
    This receives a list of questions from the manager agent and does research on them.

    ARGS: 
        queries (List[str]): List of questions from the manager agent

    RETURNS: 
        str: The compiled application section
    """
    sub_agent = assistant.create_sub_agents(name = "application_subworker")
    summaries = []
    application_links = []
    for i, query in enumerate(queries):
        print(f"Query {i+1}: {query}")
        try:
            summary = await sub_agent.ainvoke(query)
            print("Summary received:", summary)
            application_links.extend(summary.get('sources', []))
            summaries.append(f"Query {i+1} Summary:\n{summary['result']}\n")

        except Exception as e:
            summaries.append(f"Query {i+1} failed: {e}")

    prompt = ChatPromptTemplate.from_template(
  """
You are an academic writing assistant.

Your task is to synthesize the following research questions and their corresponding answers into a well-written **Application** section for a research paper.

Research Questions:
{queries}

Answers:
{summaries}

Instructions:
- Use the research questions and answers to guide the structure and content of the Application section.
- Explain how the knowledge, findings, or concepts from the research can be practically applied.
- Discuss real-world scenarios, implementations, use cases, or implications in relevant fields or industries.
- Add transitions, context, and clarifying sentences to ensure smooth flow and logical progression.
- Fill in any missing pieces using general academic or practical knowledge (without inventing specific facts).
- Maintain a clear, academic tone appropriate for a research paper.
- Do not list the questions or answers separately—integrate them into a cohesive and well-reasoned discussion.
- The final output should be a polished **Application** section that shows how the research connects to practice.

Now, write the Application section:
"""
    )
    llm = llm_worker4
    chain = prompt | llm
    application_findings = await chain.ainvoke({'queries':queries,
                               'summaries':summaries})
    application_findings = re.sub(r'<think>.*?</think>', '', application_findings.content, flags=re.DOTALL)
    return {'application_findings':application_findings, 'application_sources': application_links}
      


In [12]:
#worker that handles introduction
async def summary_worker(queries: List[str]):
    """
    This receives a list of questions from the manager agent and does research on them.

    ARGS: 
        queries (List[str]): List of questions from the manager agent

    RETURNS: 
        str: The compiled summary section
    """
    sub_agent = assistant.create_sub_agents(name="summary_subworker")
    summaries = []
    summary_links = []
    for i, query in enumerate(queries):
        print(f"Query {i+1}: {query}")
        try:
            summary = await sub_agent.ainvoke(query)
            print("Summary received:", summary)
            print("Summary received:", summary)
            summary_links.extend(summary.get('sources', []))
            summaries.append(f"Query {i+1} Summary:\n{summary['result']}\n")
        except Exception as e:
            summaries.append(f"Query {i+1} failed: {e}")

    prompt = ChatPromptTemplate.from_template(
  """
You are an academic writing assistant.

Your task is to synthesize the following research questions and their corresponding answers into a well-written **Summary** section for a research paper.

Research Questions:
{queries}

Answers:
{summaries}

Instructions:
- Provide a concise summary that encapsulates the key points of the research.
- Highlight the main questions addressed and the core insights or findings presented in the answers.
- Emphasize the significance of the research and what the reader should take away from it.
- Use transitions and context to ensure the narrative flows logically and smoothly.
- Avoid introducing new content; focus on summarizing and reinforcing what has already been covered.
- Maintain a clear, academic tone suitable for the concluding section of a research paper.
- Do not list the questions or answers separately—blend them into a coherent and integrated overview.
- The final output should be a polished **Summary** section that reflects the entire research process and outcome.

Now, write the Summary section:
"""
    )
    llm = llm_worker5
    chain = prompt | llm
    summary = await chain.ainvoke({'queries':queries,
                               'summaries':summaries})
    summary = re.sub(r'<think>.*?</think>', '', summary.content, flags=re.DOTALL)
    return {'summary_findings':summary, 'summary_sources': summary_links}
       



In [13]:
# #compilation worker
# def compilation_worker(query: str, research: str):
#     """
#     Double checks the research done by other workers. Adds whats missing
#     """
#     prompt = ChatPromptTemplate.from_template("""<ROLE>
#         You are a senior academic research compiler and editor with deep expertise in structuring and enhancing research documents.
#         </ROLE>

#         <OBJECTIVE>
#         Enhance and complete the research section below based on the original research question.
#         </OBJECTIVE>

#         <GUIDELINES>
#         1. Read the main research question carefully.
#         2. Review the current research content provided by other worker agents.
#         3. Identify missing insights, unaddressed aspects of the question, or weak transitions.
#         4. Add new content where necessary to:
#            - Address any gaps in the research.
#            - Strengthen or clarify existing information.
#            - Ensure the section flows logically and reads smoothly.
#         5. Avoid repeating information already present, unless necessary for flow or emphasis.
#         6. Ensure the writing is academically sound, concise, and informative.
#         7. If the section is an "Introduction", ensure it sets the context.
#            If it's "Background", ensure it builds historical or conceptual depth.
#            If it's "Clinical Studies" or "Application", ensure it's supported by evidence or practical relevance.
#            If it's "Conclusion", ensure it wraps up key takeaways and touches on open challenges or future work.

#         <INPUT>
#         Research Question:
#         {query}

#         Current Research Section:
#         {research}

#         <OUTPUT>
#         Return the improved and completed research section only, in clean markdown format.
#                                               """)
#     chain = prompt | compilation_model
#     compiled_output = chain.invoke({'query':query,
#                                     'research':research})
#     return compiled_output

In [14]:
async def manager_agent2(organized_dict: dict):
    """
    Orchestrates the flow of work across all worker agents.
    """
    print("Manager has started task delegation...\n")

    intro_quiz = organized_dict['organizedSearchQueries']['Introduction']['queries']
    background_quiz = organized_dict['organizedSearchQueries']['Background']['queries']
    research_quiz = organized_dict['organizedSearchQueries']['Research Findings']['queries']
    application_quiz = organized_dict['organizedSearchQueries']['Application']['queries']
    summary_quiz = organized_dict['organizedSearchQueries']['Conclusion']['queries']

    # Call sub-agents asynchronously
    Introduction_worker = await assistannt.create_sub_agents("Introduction Worker")
    introduction_answer = await Introduction_worker.ainvoke(background_quiz)

    Background_worker = await assistant.create_sub_agents("Background Worker")
    background_answer = await Background_worker.ainvoke(background_quiz)

    Research_worker = await assistant.create_sub_agents("Research Worker")
    research_findings_answer = await Research_worker.ainvoke(research_quiz)

    Application_worker = await assistant.create_sub_agents("Application Worker")
    application_answer = await Application_worker.ainvoke(application_quiz)

    Summary_worker = await assistant.create_sub_agents("Summary Worker")
    summary_answer = await Summary_worker.ainvoke(summary_quiz)

    print("Summary:", summary_answer)

    # Dummy placeholder for introduction
    introduction_answer = "Introduction section not implemented in this version."

    final_report = {
        "Introduction Section": introduction_answer,
        "Background Section": background_answer,
        "Research Findings Section": research_findings_answer,
        "Application Section": application_answer,
        "Conclusion Section": summary_answer
    }

    return final_report


In [15]:
#create the first manager agent
async def manager_agent1(organized_dict: dict):
    """
    Orchestrates the flow of work across all worker agents.
    """
    print("Manager has started task delegation...\n")
    
    intro_quiz = organized_dict['organizedSearchQueries']['Introduction']['queries']
    background_quiz = organized_dict['organizedSearchQueries']['Background']['queries']
    research_quiz = organized_dict['organizedSearchQueries']['Research Findings']['queries']
    application_quiz = organized_dict['organizedSearchQueries']['Application']['queries']
    summary_quiz = organized_dict['organizedSearchQueries']['Conclusion']['queries']

    print("sending task to Introduction worker")
    start = time.time()
    introduction_answer = await introduction_worker(intro_quiz)
    print("Received from IntroductionWorker")
    await asyncio.sleep(3)

    print("sending task to Background worker")
    background_answer = await Background_worker(background_quiz)
    print("Received from Background Worker")
    await asyncio.sleep(3)

    print("sending task to Research worker")
    research_findings_answer = await research_findings_worker(research_quiz)
    print("Received from Research Worker")
    await asyncio.sleep(3)

    print("sending task to Application worker")
    application_answer = await application_worker(application_quiz)
    print("Received from Application Worker")
    await asyncio.sleep(3)

    print("sending task to Summary worker")
    summary_answer = await summary_worker(summary_quiz)
    print("Received from Summary Worker")
    

    final_report = {
        "Introduction Section": introduction_answer['introduction_findings'],
        "Background Section": background_answer['background_findings'],
        "Research Findings Section": research_findings_answer['research_findings'],
        "Application Section": application_answer['application_findings'],
        "Conclusion Section": summary_answer['summary_findings'],
        "Sources Section": [introduction_answer['introduction_sources'], background_answer['background_sources'], 
        research_findings_answer['research_sources'], 
        application_answer['application_sources'], summary_answer['summary_sources'] ]
    }
    return final_report


In [20]:
#testing the agent
query = "Arsenal Football Club"
print("creating multiple queries")
print("*"* 90)

multiple_queries = create_multiple_queries(query)
query_id = db.save_research_queries(session_id, query, multiple_queries)
print(multiple_queries)
print("*"* 90)
print(query_id)

print("creating the research structure")
print("*"* 90)
research_structure = create_researh_structure(query, multiple_queries)
structure_id = db.save_research_structure(query_id, session_id, research_structure)
print(research_structure)
print("*"* 90)
print(structure_id)

print("passing it to manager agent")
print("*"* 90)
answer = await manager_agent1(research_structure)
report_id = db.save_final_report(session_id, structure_id, answer)
print(answer)

creating multiple queries
******************************************************************************************


{'topic': 'Arsenal Football Club', 'organizedSearchQueries': {'Introduction': {'question': 'What is the beginning of Arsenal Football Club?', 'queries': ['Arsenal Football Club financial performance and ownership impact studies', 'Arsenal FC managerial strategies and team performance correlation research', 'Historical Arsenal success factors and modern performance challenges analysis', "Arsenal ownership structure's influence on football club operations review", 'Arsenal youth academy development pathways and first-team success studies']}}}
******************************************************************************************
4bae7985-e43b-408b-b811-ba7a11326e71
creating the research structure
******************************************************************************************
{'topic': 'Arsenal Football Club', 'organizedSearchQueries': {'Introduction': {'question': 'What is the historical significance of Arsenal Football Club in English football?', 'queries': ['Historical im

In [21]:
print(answer)

{'Introduction Section': "\n\n\n**Introduction**  \n\nThe history of English football is deeply intertwined with the legacy of Arsenal Football Club, a institution that has stood as both a witness and a catalyst to the evolution of the sport over more than a century. Founded in 1886 as Dial Square, later rebranded Woolwich Arsenal, the club’s journey—from its humble beginnings in southeast London to its status as a global footballing icon—offers a unique lens through which to examine the broader social, tactical, and cultural transformations in English football. This paper explores Arsenal’s multifaceted impact, analyzing its role as a pioneer in shaping modern football culture, its historical contributions to the development of the sport, and its enduring legacy as a symbol of innovation and success.  \n\nFrom its early years as a founding member of the Football League in 1888, Arsenal has consistently been at the forefront of institutional and tactical progress. Its relocation to Hig

In [22]:

assistant.save_final_report_as_pdf(answer, query)


PDF saved at: /workspace/speech-to-text-transcription/downloads/Generated_research_for_Arsenal_Football_Club_20250702_122955_3db517.pdf


In [19]:
session_data = db.get_session_data(session_id)
print(session_data)
 

{'session': ('29376dae-371c-4cca-ac0f-cdfa2c94c57b', 'user_725', '2025-07-02 12:18:18', '2025-07-02 12:18:18', 'active', '{"purpose": "football research"}'), 'queries': [('7614aa7f-1d2d-40c6-96b6-1a58bbfcc251', '29376dae-371c-4cca-ac0f-cdfa2c94c57b', 'The Bible', '{"topic": "The Bible", "organizedSearchQueries": {"Introduction": {"question": "What is the beginning of The Bible?", "queries": ["Historical-critical analysis of The Bible\'s composition and authorship", "Theological interpretations of The Bible\'s canonical texts across denominations", "Literary analysis of narrative structure and symbolism in The Bible", "Textual criticism of ancient The Bible manuscripts and textual variants", "Cultural impact of The Bible on Western philosophy and ethics studies"]}}}', '2025-07-02 12:18:26')], 'structures': [('379b6b24-55d4-4f32-8efd-7372698cb3da', '7614aa7f-1d2d-40c6-96b6-1a58bbfcc251', '29376dae-371c-4cca-ac0f-cdfa2c94c57b', '{"topic": "The Bible", "organizedSearchQueries": {"Introduct